# Imports

In [ ]:
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import pickle
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

# Data Loading

In [2]:
# Charger le fichier Pickle
train_data_path = Path('data/train_data.pkl')
with open(train_data_path, 'rb') as f:
    data = pickle.load(f)

images = data['images']
labels = data['labels']

In [3]:
images.shape, labels.shape

((1080, 28, 28, 3), (1080, 1))

In [4]:
image_ex = images[0]
label_ex = labels[0]
image_ex.shape, label_ex.shape

((28, 28, 3), (1,))

Notre CNN nécessitera le nombre de canaux de nos images (grayscale ou RGB) et la taille de notre image :

In [5]:
nb_of_channels = image_ex.shape[2]
dimension_of_image = image_ex.shape[0]
nb_of_channels, dimension_of_image

(3, 28)

Pour connaître le nombre de classes que l'on peut prédire, on fait :

In [6]:
nb_of_classes = len(np.unique(labels.flatten()))
nb_of_classes

5

# Architecture Design & HPs

In [7]:
class CNN(nn.Module):
    def __init__(self, in_channels: int, num_classes: int, dimension_of_image: int, conv_kernel_size: int = 3):
        """
        L'architecture de ce CNN est pour le moment arbitraire, mais nous commençons avec une
        convolution (CONV), puis par un max-pooling (POOL) suivi d'une 2ième convolution avec 
        son 2ième max-pooling (POOL) et finalement notre couche complètement connectée (FC).
        
        :param in_channels: Le nombre de canaux dans notre image.
        :type in_channels: int
        :param num_classes: Le nombre de classes que l'on veut prédire. 
        :type num_classes: int
        :param dimension_of_image: La taille de l'image (ex: 28x28),
        """
        super(CNN, self).__init__()

        nb_pools = 2
        pool_kernel_size = 2
        last_conv_out_channels = 16

        dim_after_convs = dimension_of_image
        for _ in range(nb_pools):
            dim_after_convs = dim_after_convs // pool_kernel_size

        fc_dimension = last_conv_out_channels * dim_after_convs * dim_after_convs

        self.conv1 = nn.Conv2d(
            in_channels=in_channels, 
            out_channels=8, 
            kernel_size=conv_kernel_size, 
            stride=1, 
            padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=pool_kernel_size, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8, 
            out_channels=last_conv_out_channels, 
            kernel_size=conv_kernel_size, 
            stride=1, 
            padding=1
        )
        self.fc = nn.Linear(fc_dimension, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x= self.fc(x)

        return x

In [8]:
if torch.cuda.is_available(): device = 'cuda'
elif torch.mps.is_available(): device = 'mps' 
else: device = 'cpu'

Nos hyper-paramètres autres que la taille de l'entrée (`dimension_of_image`), le nombre de canaux (`nb_of_channels`) et le nombre de classes (`nb_of_classes`) :

In [9]:
num_epochs = 100
batch_size = 64
learning_rate = 0.01

# Data Splitting

On sépare nos données d'entraînement de nos données de validation :

In [10]:
valid_ratio = 0.2
X_train, X_valid, y_train, y_valid = train_test_split(images, labels.flatten(), test_size=valid_ratio, random_state=42)

On doit normaliser les données, les convertir en tenseurs et les permutter pour que les canaux soit en 2e position :

In [11]:
X_train = torch.tensor(X_train / 255, dtype=torch.float32).permute(0, 3, 1, 2)
X_valid = torch.tensor(X_valid / 255, dtype=torch.float32).permute(0, 3, 1, 2)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_valid = torch.tensor(y_valid, dtype=torch.float32)

On groupe le tout sous forme de "dataset" pour nos "dataloaders" :

In [12]:
train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)

train_dataset.train = True
valid_dataset.train = False

In [13]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)

# Model Creation & Training

In [14]:
model = CNN(in_channels=nb_of_channels, num_classes=nb_of_classes, dimension_of_image=dimension_of_image).to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [277]:
for epoch in range(1, num_epochs + 1):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Mise à jour des paramètres
        optimizer.step()

    if epoch % 10 == 0:
        print(f' ===== Epoch [{epoch}/{num_epochs}] done ===== ')

 ===== Epoch [10/100] done ===== 
 ===== Epoch [20/100] done ===== 
 ===== Epoch [30/100] done ===== 
 ===== Epoch [40/100] done ===== 
 ===== Epoch [50/100] done ===== 
 ===== Epoch [60/100] done ===== 
 ===== Epoch [70/100] done ===== 
 ===== Epoch [80/100] done ===== 
 ===== Epoch [90/100] done ===== 
 ===== Epoch [100/100] done ===== 


# Checking Accuracy (training)

In [263]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on validation data')

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        accuracy = float(num_correct) / float(num_samples) * 100
        print(f'Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%')

    model.train()

In [264]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 695/864 with accuracy 80.44%


In [265]:
check_accuracy(val_loader, model)

Checking accuracy on validation data
Got 93/216 with accuracy 43.06%
